    Wayne Nixalo  -  20 May 2017
FAI1 - Practical Deep Learning I - Week 3 HW: Kaggle StateFarm Distracted Driver Competition

#### Imports

In [1]:
import keras
import os, sys
import numpy as np
from glob import glob
from keras.optimizers import Adam
from keras.layers.core import Dense
from keras.preprocessing import image

In [3]:
# will need this to access any libraries in superdirectories
sys.path.insert(1, os.path.join(os.getcwd(), '../utils'))
import utils
from vgg16 import Vgg16

/home/wnixalo/miniconda3/envs/FAI/lib/python2.7/site-packages/theano/gpuarray/dnn.py:135: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to version 5.1.
  warnings.warn("Your cuDNN version is more recent than "
Using cuDNN version 6021 on context None
Mapped name None to device cuda: GeForce GTX 870M (0000:01:00.0)
Using Theano backend.


#### 1. Run this the First Time Only

Download the Data and get it into the right directories. Fortunately SF already organized the data, so it's just a matter of assigning path variables. NOTE: kaggle-cli needs to be set up beforhand. Also path vars must be assigned each time this notebook is run.

In [4]:
HOME_DIR = os.getcwd()
DATA_PATH = HOME_DIR + '/data'
TRAIN_PATH = DATA_PATH + '/train'
VAL_PATH = DATA_PATH + '/valid'
TEST_PATH = DATA_PATH + '/test'

In [5]:
# create the validation directories
os.mkdir(VAL_PATH)
for i in xrange(10):
    os.mkdir(VAL_PATH + '/c' + str(i))

# # another way to do this:
# %mkdir $VAL_PATH
# for i in xrange(10):
#     %mkdir $VAL_PATH/c"$i"

#### 2. Run this if you don't have an Accurate Validation Set

Grab a random permutation from the training data for validation. Do this until validation accuracy matches test accuracy. Also see: http://stackoverflow.com/questions/2632205/how-to-count-the-number-of-files-in-a-directory-using-python

In [8]:
%cd $TRAIN_PATH

VAL_PORTION = 0.2
for i in xrange(10):
    %cd c"$i"
    g = glob('*.jpg')
    number = len(g)
    shuff = np.random.permutation(g)
    for n in xrange(int(number * VAL_PORTION)):
        os.rename(shuff[n], VAL_PATH + '/c' + str(i) + '/' + shuff[n])
    % cd ..

/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c0
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c1
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c2
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c3
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c4
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c5
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c6
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c7
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c8
/home/wnixalo/Kaukasos/FAI/lesson3/data/train
/home/wnixalo/Kaukasos/FAI/lesson3/data/train/c9
/home/wnixalo/Kaukasos/FAI/lesson3/data/train


In [ ]:
# some more setup
data_path    = DATA_DIR  + '/'
train_path   = TRAIN_DIR + '/'
valid_path   = VAL_DIR   + '/'
test_path    = TEST_DIR  + '/'
results_path = DATA_DIR  + '/results/'

batch_size = 64
# no_epochs = 3

In [ ]:
# batch generator to feed data into the model
gen = image.ImageDataGenerator()
trn_batches = gen.flow_from_directory(train_path, target_size=(640,480),
                class_mode='categorical', shuffle=True, batch_size=batch_size)
val_batches = gen.flow_from_directory(valid_path, target_size=(640, 480),
                class_mode='categorical', shuffle=False, batch_size=batch_size)

In [ ]:
# load the VGG model, download its weights, and finetune it to the data
VGGModel = Vgg16()
VGGModel.pop()
for layer in VGGModel.layers: layer.trainable = False
VGGModel.add(Dense(10, activation='softmax'))
VGGModel.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# run the model until it overfits
VGGModel.optimizer.lr = 0.001
VGGModel.fit_generator(trn_batches, trn_batches.n, nb_epoch=1, verbose=1,
                       validation_data=val_batches, nb_val_samples=val_batches.n)